# Spatio-temporal mineral prospectivity modelling of the Papua New Guinea and Solomon Islands region

### Ehsan Farahbakhsh<sup>1</sup>, Sabin Zahirovic<sup>1</sup>, Brent I. A. McInnes<sup>2</sup>, Sara Polanco<sup>1</sup>, Fabian Kohlmann<sup>3</sup>, Maria Seton<sup>1</sup>, R. Dietmar M&uuml;ller<sup>1</sup>

<sup>1</sup>*EarthByte Group, School of Geosciences, The University of Sydney, Sydney, Australia*

<sup>2</sup>*John de Laeter Centre, Faculty of Science and Engineering, Curtin University, Perth, Australia*

<sup>3</sup>*Lithodat Pty. Ltd., Melbourne, Australia*

This notebook enables the user to create a spatio-temporal mineral prospectivity model of the Papua New Guinea and Solomon Islands region using two plate motion models developed by M&uuml;ller et al. (2016) and M&uuml;ller et al. (2019). It comprises two main sections; in the first section, kinematic features are extracted, and in the second section, machine learning algorithms are applied to create a prospectivity model.

### Libraries

In [ ]:
# setup the working environment
import warnings
warnings.filterwarnings('ignore')

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from collections import deque
import contextily as cx
import cv2
import geopandas as gpd
import gplately
from ipywidgets import interact
import math
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
import matplotlib.patheffects as pe
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from netCDF4 import Dataset
import numpy as np
import os
import pandas as pd
import pickle
import plotly.express as px
from ptt.subduction_convergence import subduction_convergence_over_time
from pulearn import BaggingPuClassifier
import pygplates
from scipy import ndimage, stats
from scipy.interpolate import griddata, make_interp_spline
from scipy.ndimage import gaussian_filter1d
import scipy.spatial
import seaborn as sns
import shapefile
from shapely.geometry import LineString, Point
import statistics
from tqdm.notebook import tqdm

# machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from skopt import BayesSearchCV
from skopt.space import Categorical, Integer, Real

from lib import *

# load parameters
from parameters_muller2019_v27 import parameters
# from parameters_muller2016_v27 import parameters

### Extract Convergence Kinematic Features

In [ ]:
plate_motion_model = parameters['plate_motion_model']

# get start time, end time, and time step from parameters.py
time_period = parameters['time']
start_time = time_period['start']
end_time = time_period['end']
time_step = time_period['step']
time_steps = list(range(start_time, end_time+1, time_step))

conv_dir = parameters['convergence_data_dir']
conv_prefix = parameters['convergence_data_filename_prefix']
conv_ext = parameters['convergence_data_filename_ext']

trench_points_features(
    start_time,
    end_time,
    time_step,
    conv_dir,
    conv_prefix,
    conv_ext,
    plate_motion_model,
    random_state=42
)

### Create the Plate Reconstruction Model

In [ ]:
if plate_motion_model == 'muller2016':
    rotation_files = parameters['rotation_file']
    topology_files = parameters['topology_file']
elif plate_motion_model == 'muller2019':
    rotation_files = [os.path.join(dirpath, f) for (dirpath, dirnames, filenames) in os.walk(parameters['rotation_dir']) for f in filenames]
    topology_files = [os.path.join(dirpath, f) for (dirpath, dirnames, filenames) in os.walk(parameters['topology_dir']) for f in filenames]

coastlines = parameters['coastlines_file']
static_polygons = parameters['static_polygons_file']
continents = parameters['coastlines_file']
cob = parameters['cob_file']

rotation_model = pygplates.RotationModel(rotation_files)

topology_features = pygplates.FeatureCollection()
for topology_file in topology_files:
    topology_features.add(pygplates.FeatureCollection(topology_file))

# use the PlateReconstruction object to create a plate motion model
model = gplately.PlateReconstruction(rotation_model, topology_features, static_polygons)

### Plot Kinematic Features in a Global Scale

In [ ]:
selected_features = parameters['selected_features']
selected_features_plot = selected_features.copy()
selected_features_plot.remove('distance_deg')

agegrid_dir = parameters['agegrid_dir']

extent_globe = [-180, 180, -90, 90]

@interact
def show_map(time=time_steps, feature=selected_features_plot):
    # call the PlotTopologies object
    gplot = gplately.PlotTopologies(model, coastlines, continents, cob, time=time)
    
    if plate_motion_model == 'muller2016':
        agegrid_file = agegrid_dir + f'Muller_etal_2016_AREPS_v1.17_AgeGrid-{time}.nc'
    elif plate_motion_model == 'muller2019':
        agegrid_file = agegrid_dir + f'Muller_etal_2019_Tectonics_v2.0_AgeGrid-{time}.nc'
    
    agegrid = gplately.grids.read_netcdf_grid(agegrid_file)
    
    features_t = pd.read_csv(f'{conv_dir}/{conv_prefix}_{time}.00.{conv_ext}', index_col=False)

    # dual colour bars
    fig = plt.figure(figsize=(16, 12))
    gs = GridSpec(2, 2, hspace=-0.4, wspace=0.4, height_ratios=[1, 0.02])
    ax = fig.add_subplot(gs[0, :], projection=ccrs.Mollweide(central_longitude=150))
    
    ax.set_global()

    im = gplot.plot_grid(ax, agegrid.data, cmap='viridis', vmin=0, vmax=230, alpha=0.5, zorder=1)

    gplot.plot_continents(ax, edgecolor='none', facecolor='tan', alpha=0.7, zorder=2)
    gplot.plot_ridges(ax, color='red', label='Ridge', alpha=0.5, zorder=3)
    gplot.plot_plate_motion_vectors(ax, spacingX=10, spacingY=10, normalise=True, alpha=0.1, zorder=4)

    sc = ax.scatter(features_t['trench_lon'], features_t['trench_lat'], 50, marker='.',
                    c=features_t[feature], cmap='YlOrRd', transform=ccrs.PlateCarree(), zorder=5) # cmap: Spectral_r, YlOrRd

    gplot.plot_trenches(ax, color='k', alpha=0.3, zorder=6)
    gplot.plot_subduction_teeth(ax, spacing=0.05, color='k', alpha=0.3, zorder=7)
    
    ax.gridlines(linestyle=':')
    
    cax1 = fig.add_subplot(gs[1, 0])
    cax2 = fig.add_subplot(gs[1, 1])
    
    fig.colorbar(sc, cax=cax2, orientation='horizontal', label=feature, extend='both')
    fig.colorbar(im, cax=cax1, orientation='horizontal', label='Seafloor age (Ma)', extend='max')
        
    ax.set_title(f'Subduction Zones {time} Ma')
    ax.legend(loc='lower left')
    
    plt.show()

### Clip Trench Points based on the Target Extent

In [ ]:
target_extent_file = parameters['target_extent_file']
target_extent_gdf = gpd.read_file(target_extent_file)
target_extent_bounds = target_extent_gdf.bounds
target_extent = [target_extent_bounds.loc[0]['minx'], target_extent_bounds.loc[0]['maxx'],
                 target_extent_bounds.loc[0]['miny'], target_extent_bounds.loc[0]['maxy']]

if target_extent[0] < -180:
    target_extent[0] = -180
if target_extent[1] > 180:
    target_extent[1] = 180
if target_extent[2] < -90:
    target_extent[2] = -90
if target_extent[3] > 90:
    target_extent[3] = 90

features_target_extent_files_lst = []

for time in time_steps:
    features_target_extent_files_lst.append(f'{conv_dir}/{conv_prefix}_target_extent_{time}.00.{conv_ext}')

for features_target_extent_file in tqdm(features_target_extent_files_lst):
    if not os.path.isfile(features_target_extent_file):
        time = features_target_extent_files_lst.index(features_target_extent_file)
        features_t = pd.read_csv(f'{conv_dir}/{conv_prefix}_{time}.00.{conv_ext}', index_col=False)
        features_target_extent_lst = []

        for i in range(features_t.shape[0]):
            x = features_t.iloc[i]['trench_lon']
            y = features_t.iloc[i]['trench_lat']
            p = Point((x, y))
            if p.within(target_extent_gdf.geometry[0]):
                features_target_extent_lst.append(features_t.iloc[i].values)

        features_target_extent = pd.DataFrame(np.row_stack(features_target_extent_lst), columns=features_t.columns)
        features_target_extent.to_csv(features_target_extent_file, index=False)

### Plot Kinematic Features based on the Target Extent

In [ ]:
def set_ax(ax, extent, interval_x, interval_y, font_size=None, stock_img=True, order=None):
    if stock_img:
        ax.stock_img()

    ax.set_extent(extent)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=1, color='gray', alpha=0.5, linestyle='--', 
                      xlocs=np.arange(-180, 180, interval_x), ylocs=np.arange(-90, 90, interval_y), zorder=order)
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'color': 'gray', 'weight': 'bold', 'fontsize': font_size}
    gl.ylabel_style = {'color': 'gray', 'weight': 'bold', 'fontsize': font_size}
    
proj = ccrs.LambertAzimuthalEqualArea(150, 0)

@interact
def show_map(time=time_steps, feature=selected_features_plot):
    # call the PlotTopologies object
    gplot = gplately.PlotTopologies(model, coastlines, continents, cob, time=time)
    
    if plate_motion_model == 'muller2016':
        agegrid_file = agegrid_dir + f'Muller_etal_2016_AREPS_v1.17_AgeGrid-{time}.nc'
    elif plate_motion_model == 'muller2019':
        agegrid_file = agegrid_dir + f'Muller_etal_2019_Tectonics_v2.0_AgeGrid-{time}.nc'
    
    agegrid = gplately.grids.read_netcdf_grid(agegrid_file)
    
    features_t = pd.read_csv(f'{conv_dir}/{conv_prefix}_target_extent_{time}.00.{conv_ext}', index_col=False)

    # dual colour bars
    fig = plt.figure(figsize=(6, 8))
    gs = GridSpec(2, 2, hspace=-0.1, wspace=0.1, height_ratios=[1, 0.02])
    ax = fig.add_subplot(gs[0, :], projection=proj)
    
    set_ax(ax, target_extent, 15, 15, stock_img=False, order=8)

    im = gplot.plot_grid(ax, agegrid.data, cmap='viridis', vmin=0, vmax=230, alpha=0.5, zorder=1)

    gplot.plot_continents(ax, edgecolor='none', facecolor='tan', alpha=0.7, zorder=2)
    gplot.plot_ridges(ax, color='red', label='Ridge', alpha=0.5, zorder=3)
    gplot.plot_plate_motion_vectors(ax, spacingX=30, spacingY=30, normalise=False, regrid_shape=20, alpha=0.2, zorder=4)

    sc = ax.scatter(features_t['trench_lon'], features_t['trench_lat'], 50, marker='.',
                    c=features_t[feature], cmap='YlOrRd', transform=ccrs.PlateCarree(), zorder=5)

    gplot.plot_trenches(ax, color='k', alpha=0.3, zorder=6)
    gplot.plot_subduction_teeth(ax, spacing=0.03, color='k', alpha=0.3, zorder=7)
    
    cax1 = fig.add_subplot(gs[1, 0])
    cax2 = fig.add_subplot(gs[1, 1])
    
    fig.colorbar(sc, cax=cax2, orientation='horizontal', label=feature, extend='both')
    fig.colorbar(im, cax=cax1, orientation='horizontal', label='Seafloor age (Ma)', extend='max')
    
    ax.legend(loc='lower left')
    
    plt.show()

### Reconstruct Mineral Occurrences

In [ ]:
min_occ_file = parameters['min_occ_file']
coreg_input_dir = parameters['coreg_input_dir']
coreg_input_files = parameters['coreg_input_files']
min_occ_data_file = coreg_input_dir + coreg_input_files[0] # returns 'mineral_occurrences'

if os.path.isfile(min_occ_data_file):
    min_occ_data = pd.read_csv(min_occ_data_file, index_col=False)
else:
    # id, lon, lat, age, and plate id of mineral occurrences
    min_occ_data = process_real_deposits(min_occ_file, start_time, end_time, time_step, plate_motion_model)
    # save the attributes of mineral occurrences
    min_occ_data.to_csv(min_occ_data_file, index=False)

In [ ]:
proj = ccrs.LambertAzimuthalEqualArea(150, 0)

@interact
def show_map(time=time_steps, feature=selected_features_plot):
    lons_lats_recon = []
    
    for min_occ in min_occ_data.iterrows():
        if time == 0:
            lons_lats_recon.append((min_occ[1]['lon'], min_occ[1]['lat']))
        elif int(min_occ[1]['age']) < time:
            lons_lats_recon.append((np.nan, np.nan))
        elif int(min_occ[1]['age']) == time:
            lons_lats_recon.append((min_occ[1]['lon_recon'], min_occ[1]['lat_recon']))
        else:
            lat_lon_recon = get_recon_ccords([min_occ[1]['lon']],
                                             [min_occ[1]['lat']],
                                             plate_motion_model='muller2019', # assign 'muller2019' or 'muller2016'
                                             time=time)[0]
            lons_lats_recon.append(tuple((lat_lon_recon[1], lat_lon_recon[0])))

    # call the PlotTopologies object
    gplot = gplately.PlotTopologies(model, coastlines, continents, cob, time=time)
    
    if plate_motion_model == 'muller2016':
        agegrid_file = agegrid_dir + f'Muller_etal_2016_AREPS_v1.17_AgeGrid-{time}.nc'
    elif plate_motion_model == 'muller2019':
        agegrid_file = agegrid_dir + f'Muller_etal_2019_Tectonics_v2.0_AgeGrid-{time}.nc'
    
    agegrid = gplately.grids.read_netcdf_grid(agegrid_file)
    
    features_t = pd.read_csv(f'{conv_dir}/{conv_prefix}_{time}.00.{conv_ext}', index_col=False)

    # dual colour bars
    fig = plt.figure(figsize=(6, 8))
    gs = GridSpec(2, 2, hspace=-0.1, wspace=0.1, height_ratios=[1, 0.02])
    ax = fig.add_subplot(gs[0, :], projection=proj)
    
    set_ax(ax, target_extent, 15, 15, stock_img=False, order=9)

    im = gplot.plot_grid(ax, agegrid.data, cmap='viridis', vmin=0, vmax=230, alpha=0.5, zorder=1)

    gplot.plot_continents(ax, edgecolor='none', facecolor='tan', alpha=0.7, zorder=2)
    gplot.plot_ridges(ax, color='red', label='Ridge', alpha=0.5, zorder=3)
    gplot.plot_plate_motion_vectors(ax, spacingX=20, spacingY=20, normalise=False, regrid_shape=20, alpha=0.2, zorder=4)

    sc0 = ax.scatter(features_t['trench_lon'], features_t['trench_lat'], 50, marker='.',
                    c=features_t[feature], cmap='YlOrRd', transform=ccrs.PlateCarree(), zorder=5)
    
    sc1 = ax.scatter(
        [coords[0] for coords in lons_lats_recon],
        [coords[1] for coords in lons_lats_recon],
        transform=ccrs.PlateCarree(),
        marker='X',
        facecolor='yellow',
        edgecolor='black',
        s=50,
        alpha=0.7,
        zorder=6
    )

    gplot.plot_trenches(ax, color='k', alpha=0.3, zorder=7)
    gplot.plot_subduction_teeth(ax, spacing=0.03, color='k', alpha=0.3, zorder=8)
    
    cax1 = fig.add_subplot(gs[1, 0])
    cax2 = fig.add_subplot(gs[1, 1])
    
    fig.colorbar(sc0, cax=cax2, orientation='horizontal', label=feature, extend='both')
    fig.colorbar(im, cax=cax1, orientation='horizontal', label='Seafloor age (Ma)', extend='max')
    
    ax.legend(loc='lower left')
    
    plt.show()

### Create Buffer Zones

In [ ]:
if plate_motion_model == 'muller2016':
    rotation_files = parameters['rotation_files']
    topology_files = parameters['topology_files']
elif plate_motion_model == 'muller2019':
    rotation_files = [os.path.join(dirpath, f) for (dirpath, dirnames, filenames) in os.walk(parameters['rotation_dir']) for f in filenames]
    topology_files = [os.path.join(dirpath, f) for (dirpath, dirnames, filenames) in os.walk(parameters['topology_dir']) for f in filenames]

buffer_zones_files_lst = []

for time in time_steps:
    buffer_zones_files_lst.append(f'{coreg_input_dir}buffer_zones/buffer_zone_{time}_Ma.shp')

for buffer_zone_file in tqdm(buffer_zones_files_lst):
    if not os.path.isfile(buffer_zone_file):
        index = buffer_zones_files_lst.index(buffer_zone_file)
        
        resolved_topologies = []
        shared_boundary_sections = []
        # use pygplates to resolve the topologies
        pygplates.resolve_topologies(topology_files, rotation_files, resolved_topologies, time_steps[index], shared_boundary_sections)

        # subduction zones
        subduction_geoms = []
        get_subduction_geometries(subduction_geoms, shared_boundary_sections)

        _, buffer_zone = generate_buffer_zones(subduction_geoms, width=3)
        buffer_zone.to_file(buffer_zone_file)
        print(f'Buffer zones saved to {buffer_zone_file}')

buffer_zones_lst = []
buffer_zones_clipped_lst = []

for buffer_zone_file in buffer_zones_files_lst:
    buffer_zone = gpd.read_file(buffer_zone_file)
    buffer_zone_clipped = buffer_zone.clip(target_extent_gdf)
    buffer_zones_lst.append(buffer_zone)
    buffer_zones_clipped_lst.append(buffer_zone_clipped)

### Plot Buffer Zones in a Global Scale

In [ ]:
proj = ccrs.Mollweide(central_longitude=150)

@interact
def show_map(time=time_steps):
    # call the PlotTopologies object
    gplot = gplately.PlotTopologies(model, coastlines, continents, cob, time=time)
    
    if plate_motion_model == 'muller2016':
        agegrid_file = agegrid_dir + f'Muller_etal_2016_AREPS_v1.17_AgeGrid-{time}.nc'
    elif plate_motion_model == 'muller2019':
        agegrid_file = agegrid_dir + f'Muller_etal_2019_Tectonics_v2.0_AgeGrid-{time}.nc'
    
    agegrid = gplately.grids.read_netcdf_grid(agegrid_file)
    
    # single colour bar
    fig = plt.figure(figsize=(16, 12))
    ax = plt.axes(projection=proj)
    ax.set_global()

    im = gplot.plot_grid(ax, agegrid.data, cmap='viridis', vmin=0, vmax=230, alpha=0.5, zorder=1)

    gplot.plot_continents(ax, edgecolor='none', facecolor='tan', alpha=0.7, zorder=2)
    gplot.plot_ridges(ax, color='red', label='Ridge', alpha=0.5, zorder=3)
    gplot.plot_plate_motion_vectors(ax, spacingX=10, spacingY=10, normalise=True, alpha=0.1, zorder=4)

    buffer_zones_lst[time_steps.index(time)].plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        edgecolor='none',
        facecolor='gray',
        linewidth=1,
        alpha=0.7,
        zorder=5,
    )
    
    gplot.plot_trenches(ax, color='k', alpha=1, zorder=6)
    gplot.plot_subduction_teeth(ax, spacing=0.05, color='k', alpha=1, zorder=7)
    
    ax.gridlines(linestyle=':')
    
    fig.colorbar(im, orientation='horizontal', shrink=0.4, pad=0.05, label='Seafloor Age (Ma)', extend='max')
    
    ax.legend(loc='lower left')
    
    plt.show()

### Plot Buffer Zones based on the Target Extent

In [ ]:
proj = ccrs.LambertAzimuthalEqualArea(150, 0)

@interact
def show_map(time=time_steps):
    # call the PlotTopologies object
    gplot = gplately.PlotTopologies(model, coastlines, continents, cob, time=time)
    
    if plate_motion_model == 'muller2016':
        agegrid_file = agegrid_dir + f'Muller_etal_2016_AREPS_v1.17_AgeGrid-{time}.nc'
    elif plate_motion_model == 'muller2019':
        agegrid_file = agegrid_dir + f'Muller_etal_2019_Tectonics_v2.0_AgeGrid-{time}.nc'
    
    agegrid = gplately.grids.read_netcdf_grid(agegrid_file)
    
    # single colour bar
    fig = plt.figure(figsize=(6, 8))
    ax = plt.axes(projection=proj)
    
    set_ax(ax, target_extent, 15, 15, stock_img=False, order=8)

    im = gplot.plot_grid(ax, agegrid.data, cmap='viridis', vmin=0, vmax=230, alpha=0.5, zorder=1)

    gplot.plot_continents(ax, edgecolor='none', facecolor='tan', alpha=0.7, zorder=2)
    gplot.plot_ridges(ax, color='red', label='Ridge', alpha=0.5, zorder=3)
    gplot.plot_plate_motion_vectors(ax, spacingX=20, spacingY=20, normalise=False, regrid_shape=20, alpha=0.2, zorder=4)
    
    buffer_zones_clipped_lst[time_steps.index(time)].plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        edgecolor='none',
        facecolor='gray',
        linewidth=1,
        alpha=0.7,
        zorder=5
    )
    
    gplot.plot_trenches(ax, color='k', alpha=0.3, zorder=6)
    gplot.plot_subduction_teeth(ax, spacing=0.03, color='k', alpha=0.3, zorder=7)
    
    fig.colorbar(im, orientation='horizontal', shrink=0.4, pad=0.05, label='Seafloor Age (Ma)', extend='max')
    
    ax.legend(loc='lower left')

    plt.show()

### Generate Random Samples

In [ ]:
random_data_file = coreg_input_dir + coreg_input_files[1]
num_features = len(selected_features)

if os.path.isfile(random_data_file):
    random_data = pd.read_csv(random_data_file, index_col=False)
    time_steps_random = random_data['age'].tolist()
else:
    time_steps_random, random_data = generate_random_samples(buffer_zones_clipped_lst,
                                          start_time=start_time,
                                          end_time=end_time,
                                          time_step=time_step,
                                          num_features=num_features,
                                          num_features_factor=10,
                                          rand_factor=20,
                                          plate_motion_model='muller2019', # assign 'muller2019' or 'muller2016'
                                          random_state=42
                                         )
    random_data.to_csv(random_data_file, index=False, float_format='%.4f')

### Plot Random Samples based on the Target Extent

In [ ]:
time_steps_random_sorted = sorted(time_steps_random)
time_steps_random_sorted = [*set(time_steps_random_sorted)]

proj = ccrs.LambertAzimuthalEqualArea(150, 0)

@interact
def show_map(time=time_steps_random_sorted):
    # call the PlotTopologies object
    gplot = gplately.PlotTopologies(model, coastlines, continents, cob, time=time)
    
    if plate_motion_model == 'muller2016':
        agegrid_file = agegrid_dir + f'Muller_etal_2016_AREPS_v1.17_AgeGrid-{time}.nc'
    elif plate_motion_model == 'muller2019':
        agegrid_file = agegrid_dir + f'Muller_etal_2019_Tectonics_v2.0_AgeGrid-{time}.nc'
    
    agegrid = gplately.grids.read_netcdf_grid(agegrid_file)
    
    # single colour bar
    fig = plt.figure(figsize=(6, 8))
    ax = plt.axes(projection=proj)
    
    set_ax(ax, target_extent, 15, 15, stock_img=False, order=9)

    im = gplot.plot_grid(ax, agegrid.data, cmap='viridis', vmin=0, vmax=230, alpha=0.5, zorder=1)

    gplot.plot_continents(ax, edgecolor='none', facecolor='tan', alpha=0.7, zorder=2)
    gplot.plot_ridges(ax, color='red', label='Ridge', alpha=0.5, zorder=3)
    gplot.plot_plate_motion_vectors(ax, spacingX=20, spacingY=20, normalise=False, regrid_shape=20, alpha=0.2, zorder=4)
    
    buffer_zones_clipped_lst[time_steps.index(time)].plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        edgecolor='none',
        facecolor='gray',
        linewidth=1,
        alpha=0.7,
        zorder=5
    )
    
    random_samples = random_data.loc[random_data['age'] == time]
    ax.scatter(
        random_samples['lon'],
        random_samples['lat'],
        transform=ccrs.PlateCarree(),
        marker='X',
        edgecolor='black',
        facecolor='cyan',
        s=50,
        zorder=8
    )
    
    gplot.plot_trenches(ax, color='k', alpha=0.3, zorder=6)
    gplot.plot_subduction_teeth(ax, spacing=0.03, color='k', alpha=0.3, zorder=7)
    
    fig.colorbar(im, orientation='horizontal', shrink=0.4, pad=0.05, label='Seafloor Age (Ma)', extend='max')
    
    ax.legend(loc='lower left')
        
    plt.show()

### Generate Target Points

In [ ]:
target_points_coreg_in_files_lst = []
mask_coords_files_lst = []

for time in time_steps:
    target_points_coreg_in_files_lst.append(coreg_input_dir + coreg_input_files[2] + f'_{time}_Ma.csv')
    mask_coords_files_lst.append(coreg_input_dir + f'mask_{time}_Ma.csv')

for target_points_file, mask_coords_file in tqdm(zip(target_points_coreg_in_files_lst, mask_coords_files_lst), total=len(target_points_coreg_in_files_lst)):
    if not(os.path.isfile(target_points_file) and os.path.isfile(mask_coords_file)):
        index = target_points_coreg_in_files_lst.index(target_points_file)

        # generate target points
        target_points, mask_coords, nx, ny = generate_samples(buffer_zones_clipped_lst[index], 0.5, 0.5, # dist_x and dist_y
                                                              time_steps[index], plate_motion_model='muller2019') # assign 'muller2019' or 'muller2016'
        # save the attributes of target points
        target_points.to_csv(target_points_file, index=False, float_format='%.4f')
        # save the mask
        mask_coords.to_csv(mask_coords_file, index=False, float_format='%.4f')
        print(f'Target points saved to {target_points_file}')

target_points_coreg_in_lst = []
mask_coords_lst = []

for target_points_file, mask_coords_file in zip(target_points_coreg_in_files_lst, mask_coords_files_lst):
    target_points_coreg_in_lst.append(pd.read_csv(target_points_file, index_col=False))
    mask_coords_lst.append(pd.read_csv(mask_coords_file, index_col=False))

### Plot Target Points based on the Target Extent

In [ ]:
proj = ccrs.LambertAzimuthalEqualArea(150, 0)

@interact
def show_map(time=time_steps):
    # call the PlotTopologies object
    gplot = gplately.PlotTopologies(model, coastlines, continents, cob, time=time)
    
    if plate_motion_model == 'muller2016':
        agegrid_file = agegrid_dir + f'Muller_etal_2016_AREPS_v1.17_AgeGrid-{time}.nc'
    elif plate_motion_model == 'muller2019':
        agegrid_file = agegrid_dir + f'Muller_etal_2019_Tectonics_v2.0_AgeGrid-{time}.nc'

    agegrid = gplately.grids.read_netcdf_grid(agegrid_file)
    
    # single colour bar
    fig = plt.figure(figsize=(6, 8))
    ax = plt.axes(projection=proj)
    
    set_ax(ax, target_extent, 15, 15, stock_img=False, order=9)

    im = gplot.plot_grid(ax, agegrid.data, cmap='viridis', vmin=0, vmax=230, alpha=0.5, zorder=1)

    gplot.plot_continents(ax, edgecolor='none', facecolor='tan', alpha=0.7, zorder=2)
    gplot.plot_ridges(ax, color='red', label='Ridge', alpha=0.5, zorder=3)
    gplot.plot_plate_motion_vectors(ax, spacingX=20, spacingY=20, normalise=False, regrid_shape=20, alpha=0.2, zorder=4)
    
    buffer_zones_clipped_lst[time_steps.index(time)].plot(
        ax=ax,
        transform=ccrs.PlateCarree(),
        edgecolor='none',
        facecolor='gray',
        linewidth=1,
        alpha=0.7,
        zorder=5
    )
    
    ax.scatter(
        target_points_coreg_in_lst[time_steps.index(time)]['lon'],
        target_points_coreg_in_lst[time_steps.index(time)]['lat'],
        transform=ccrs.PlateCarree(),
        marker='.',
        c='red',
        s=1,
        zorder=8
    )
        
    gplot.plot_trenches(ax, color='k', alpha=0.3, zorder=6)
    gplot.plot_subduction_teeth(ax, spacing=0.03, color='k', alpha=0.3, zorder=7)
    
    fig.colorbar(im, orientation='horizontal', shrink=0.4, pad=0.05, label='Seafloor Age (Ma)', extend='max')
    
#     plt.savefig(
#     f'./figures/muller2019/target_points.png',
#     bbox_inches='tight',
#     pad_inches=0.1,
#     dpi=150
#     )

    ax.legend(loc='lower left')
        
    plt.show()

# Coregistration and Data Wrangling

In [ ]:
coreg_output_dir = parameters['coreg_output_dir']
positive_data_file = coreg_output_dir + coreg_input_files[0]
unlabelled_data_file = coreg_output_dir + coreg_input_files[1]
target_points_coreg_out_files_lst = []

for time in time_steps:
    target_points_coreg_out_files_lst.append(coreg_output_dir + coreg_input_files[2] + f'_{time}_Ma.csv')

coregistration(
    coreg_input_dir,
    coreg_output_dir,
    coreg_input_files,
    conv_dir,
    conv_prefix,
    conv_ext,
    time_steps=time_steps,
    search_radius=3
)

positive_data = pd.read_csv(positive_data_file, index_col=False)
unlabelled_data = pd.read_csv(unlabelled_data_file, index_col=False)

target_points_coreg_out_lst = []
for file_name in target_points_coreg_out_files_lst:
    target_points_coreg_out_lst.append(pd.read_csv(file_name, index_col=False))

In [ ]:
ml_input_dir = parameters['ml_input_dir']

positive_data['label'] = 1
unlabelled_data['label'] = 0

positive_features = positive_data[selected_features]
unlabelled_features = unlabelled_data[selected_features]
features_all = pd.concat([positive_features, unlabelled_features]).reset_index(drop=True)

# --------------------
# save correlation csv file
corr_file = ml_input_dir + 'correlation.csv'

if os.path.isfile(corr_file):
    corr = pd.read_csv(corr_file, index_col=0)
else:
    corr = features_all.corr(method='spearman').round(3)
    corr.to_csv(corr_file, index=True)
    
# --------------------
corr.style.background_gradient(cmap='coolwarm', axis=None).format('{:.3}')

In [ ]:
selected_features_names = parameters['selected_features_names_nounit']
corr.columns = selected_features_names
corr.index = selected_features_names
f = plt.figure(figsize=(20, 15))
plt.matshow(corr, fignum=f.number, cmap='coolwarm', vmin=-1, vmax=1)
plt.xticks(range(corr.select_dtypes(['number']).shape[1]), corr.select_dtypes(['number']).columns, fontsize=14, rotation=90)
plt.yticks(range(corr.select_dtypes(['number']).shape[1]), corr.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar(aspect=50)
cb.ax.tick_params(labelsize=14)
# plt.savefig(
#     f'./figures/muller2019/correlation.png',
#     bbox_inches='tight',
#     pad_inches=0.1,
#     dpi=150
#     )

In [ ]:
for column in corr.columns:
    positive_corr_lst = []
    negative_corr_lst = []
    feature = corr[column]
    for i in range(feature.shape[0]):
        if feature[i] > 0.7:
            positive_corr_lst.append(feature.index[i])
        elif feature[i] < -0.7:
            negative_corr_lst.append(feature.index[i])
    positive_corr_lst.remove(column)
    if positive_corr_lst:
        print(f'{column} is positively correlated with', end=' ')
        print(*positive_corr_lst, sep=', ')
    if negative_corr_lst:
        print(f'{column} is negatively correlated with', end=' ')
        print(*negative_corr_lst, sep=', ')
    if positive_corr_lst or negative_corr_lst:
        print('-----')

In [ ]:
augmentation_dir = parameters['augmentation_dir']

if os.path.isfile(ml_input_dir + 'features_labels_original.csv'):
    Xy_train_original_df = pd.read_csv(ml_input_dir + 'features_labels_original.csv', index_col=False)
    features_list = Xy_train_original_df.columns.tolist()
    features_list.remove('label')
    print('Training data file already exists!')
else:
    positive_labels = positive_data[positive_data.columns[-1]]
    unlabelled_labels = unlabelled_data[unlabelled_data.columns[-1]]
    labels = pd.concat([positive_labels, unlabelled_labels]).reset_index(drop=True)
    features_labels_original = pd.concat([features_all, labels], axis=1).reset_index(drop=True)

    # drop highly correlated features
    # create a correlation matrix
    corr_matrix = features_all.corr(method='spearman').abs()
    # select the upper triangle of the correlation matrix
    corr_upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    # find features with the correlation greater than 0.7
    corr_drop = [column for column in corr_upper.columns if any(corr_upper[column] > 0.7)]
    print('List of the features removed due to high correlation with other features:', corr_drop)
    # drop features
    features = features_all.drop(corr_drop, axis=1)
    features_list = features.columns.tolist()
    features_labels = pd.concat([features, labels], axis=1).reset_index(drop=True)
    features_labels_list = features_list.copy()
    features_labels_list.append('label')

    positive_data = features_labels[features_labels['label']==1]
    unlabelled_data = features_labels[features_labels['label']==0]

    positive_features = positive_data[positive_data.columns[:-1]]
    unlabelled_features = unlabelled_data[unlabelled_data.columns[:-1]]

    positive_labels = positive_data[positive_data.columns[-1]]
    unlabelled_labels = unlabelled_data[unlabelled_data.columns[-1]]

    # train test
    # split positive samples into training and test datasets
    X_train, X_test, y_train, y_test = train_test_split(positive_features, positive_labels, train_size=0.8, random_state=42)
    X_train = np.vstack((X_train, unlabelled_features))
    y_train = np.vstack((y_train.values.reshape(-1, 1), unlabelled_labels.values.reshape(-1, 1)))
    Xy_train_original = np.hstack((X_train, y_train))
    Xy_train_original_df = pd.DataFrame(Xy_train_original, columns=features_labels_list)
    Xy_train_original_df.to_csv(ml_input_dir + 'features_labels_original.csv', index=False)

    print('\nNumber of the features reduced from', num_features, 'to', positive_features.shape[1])
    print('Number of the positive samples:', positive_features.shape[0])

    print('\nNumber of the training samples:', X_train.shape[0]), print('Number of the training labels:', y_train.shape[0])
    print('Number of the testing samples:', X_test.shape[0]), print('Number of the testing labels:', y_test.shape[0])

    st_scaler = StandardScaler()
    X_train = st_scaler.fit_transform(X_train)
    X_test = st_scaler.transform(X_test)
    Xy_train = np.hstack((X_train, y_train))
    Xy_test = np.hstack((X_test, np.reshape(y_test, (-1, 1))))
    Xy_train_df = pd.DataFrame(Xy_train, columns=features_labels_list)
    Xy_test_df = pd.DataFrame(Xy_test, columns=features_labels_list)

    if plate_motion_model == 'muller2016':
        Xy_train_df.to_csv(augmentation_dir + 'Xy_train_muller2016.csv', index=False)
        Xy_test_df.to_csv(augmentation_dir + 'Xy_test_muller2016.csv', index=False)
    elif plate_motion_model == 'muller2019':
        Xy_train_df.to_csv(augmentation_dir + 'Xy_train_muller2019.csv', index=False)
        Xy_test_df.to_csv(augmentation_dir + 'Xy_test_muller2019.csv', index=False)

    # save the standard scaler model
    with open(ml_input_dir + 'st_scaler.pkl', 'wb') as f:
        pickle.dump(st_scaler, f)

In [ ]:
target_points_ml_in_files_lst = []
for time in time_steps:
    target_points_ml_in_files_lst.append(ml_input_dir + coreg_input_files[2] + f'_{time}_Ma.csv')
    
target_points_ml_in_lst = []
for target_points_ml_in_file in target_points_ml_in_files_lst:
    if os.path.isfile(target_points_ml_in_file):
        target_points_ml_in_lst.append(pd.read_csv(target_points_ml_in_file, index_col=False))
    if not os.path.isfile(target_points_ml_in_file):
        index = target_points_ml_in_files_lst.index(target_points_ml_in_file)
        target_points_ml_in = target_points_coreg_out_lst[index][selected_features]
        target_points_ml_in = target_points_ml_in[target_points_ml_in.columns.intersection(features_list)]
        
        try:
            target_points_ml_in = st_scaler.transform(target_points_ml_in)
        except:
            # load the model
            with open(ml_input_dir + 'st_scaler', 'rb') as f:
                st_scaler = pickle.load(f)
            target_points_ml_in = st_scaler.transform(target_points_ml_in)
            
        target_points_ml_in = pd.DataFrame(target_points_ml_in, columns=features_list)
        target_points_ml_in_lst.append(target_points_ml_in)
        target_points_ml_in.to_csv(target_points_ml_in_file, index=False)

# Machine Learning

In [ ]:
augmentation_dir = parameters['augmentation_dir']
ml_output_dir = parameters['ml_output_dir']

# n_estimators: number of trees in the foreset
# max_features: max number of features considered for splitting a node
# max_depth: max number of levels in each decision tree
# min_samples_split: min number of data points placed in a node before the node is split
# min_samples_leaf: min number of data points allowed in a leaf node
# bootstrap: method for sampling data points (with or without replacement)

model_file = ml_output_dir + 'model_muller2019.pkl'

if os.path.isfile(model_file):
    print('The model already exists!')
    # load the model
    with open(model_file, 'rb') as f:
        bc_best = pickle.load(f)
    Xy_train_df = pd.read_csv(ml_output_dir + 'Xy_train_muller2019.csv', index_col=False)
    
    Xy_test = pd.read_csv(augmentation_dir + 'Xy_test_muller2019.csv', index_col=False)
    X_test = Xy_test[Xy_test.columns[:-1]]
    y_test = Xy_test[Xy_test.columns[-1]]
    X_pred = bc_best.predict(X_test)
    X_pred_acc = accuracy_score(y_test, X_pred)
    print('The accuracy (testing set):', X_pred_acc)
    
    estimators = bc_best.estimators_
    importances = [estimators[j].feature_importances_.reshape(-1, 1) for j in range(len(estimators))]
    importances = np.hstack(importances)
    
    print(bc_best)
else:
    Xy_test = pd.read_csv(augmentation_dir + 'Xy_test_muller2019.csv', index_col=False)
    X_test = Xy_test[Xy_test.columns[:-1]]
    y_test = Xy_test[Xy_test.columns[-1]]
    
    # Random Forest model structure
    rf = RandomForestClassifier(n_jobs=-1, random_state=42)
    # bc = BaggingPuClassifier(rf, max_samples=int(sum(y_train)), n_jobs=-1, random_state=1)
    bc = BaggingPuClassifier(rf, n_jobs=-1, random_state=42)

    n_iter = 10
    n_fold = 10
    acc_best = 0

    for i in range(n_iter):
        print('--------------------')
        print(f'Iteration {i+1}')
        print('--------------------')

        smote_gan_file = f'./augmentation/smote_gan_muller2019_{i+1}.csv'
        smote_gan = pd.read_csv(smote_gan_file, index_col=False)
        features = smote_gan[smote_gan.columns[:-1]]
        labels = smote_gan[smote_gan.columns[-1]]
        X_train_, X_test_, y_train_, y_test_ = train_test_split(features, labels, train_size=0.8, random_state=42)
        Xy_train_ = np.hstack((X_train_, y_train_.values.reshape(-1, 1)))
        Xy_train_df_ = pd.DataFrame(Xy_train_, columns=smote_gan.columns)
        y_train_.replace(2, 1, inplace=True)
        y_test_.replace(2, 1, inplace=True)

        search_space = {
        'base_estimator__bootstrap': Categorical([True, False]), # values for boostrap can be either True or False
        'base_estimator__max_depth': Integer(5, 20), # values of max_depth are integers
        'base_estimator__max_features': Categorical([None, 'sqrt','log2']), 
        'base_estimator__min_samples_leaf': Integer(2, 20),
        'base_estimator__min_samples_split': Integer(2, 30),
        'base_estimator__n_estimators': Integer(10, 200),
        'max_samples': Integer(int(0.5*(len(y_train_)-sum(y_train_))), int(0.9*(len(y_train_)-sum(y_train_))))
        }

        bc_bayes_search = BayesSearchCV(bc, search_space, n_iter=50, # specify how many iterations
                                        scoring='accuracy', n_jobs=-1, cv=n_fold, verbose=1, random_state=42, return_train_score=True)
        bc_bayes_search.fit(X_train_, y_train_) # callback=on_step will print score after each iteration

        X_pred = bc_bayes_search.best_estimator_.predict(X_test)
        X_pred_acc = accuracy_score(y_test, X_pred)

        X_pred_ = bc_bayes_search.best_estimator_.predict(X_test_)
        X_pred_acc_ = accuracy_score(y_test_, X_pred_)

        if X_pred_acc > acc_best:
            acc_best = X_pred_acc
            bc_best_acc_test = X_pred_acc.copy()
            
            bc_best = bc_bayes_search.best_estimator_
            bc_best_acc_ = bc_bayes_search.best_score_
            bc_best_acc_test_ = X_pred_acc_.copy()
            
            estimators = bc_best.estimators_
            importances = [estimators[j].feature_importances_.reshape(-1, 1) for j in range(len(estimators))]
            importances = np.hstack(importances)
            
            Xy_train_df_.to_csv(ml_output_dir + 'Xy_train_muller2019.csv', index=False)
            print('\nIteration number with the highest accuracy:', i+1)
            print('The accuracy (testing set):', bc_best_acc_test)
        
    print('\nThe highest accuracy during cross validation:', bc_best_acc)
    print('The highest accuracy (validation set):', bc_best_acc_test_)
    print('The highest accuracy (testing set):', bc_best_acc_test)
    print('\n')
    print(bc_best)
    
    # save the model
    with open(model_file, 'wb') as f:
        pickle.dump(bc_best, f)

In [ ]:
labels_estimated = []
for pair in bc_best.oob_decision_function_:
    if np.isnan(pair[0]) or pair[0] < pair[1]:
        labels_estimated.append(2)
    else:
        labels_estimated.append(0)
        
print('Number of positive samples', labels_estimated.count(2))
print('Number of negative samples', labels_estimated.count(0))

In [ ]:
feature_importances_file = ml_output_dir + 'feature_importances_muller2019.csv'

if os.path.isfile(feature_importances_file):
    feature_importances = pd.read_csv(feature_importances_file, index_col=False).to_numpy().tolist()
else:
    output_features = Xy_train_original_df.columns.tolist()
    output_features.remove('label')
    output_features_index = [selected_features.index(feature) for feature in output_features]
    selected_features_names = parameters['selected_features_names_nounit']
    selected_features_names = [selected_features_names[i] for i in output_features_index]

    importances_mean = importances.mean(axis=1)
    importances_var = importances.var(axis=1)

    feature_importances = [(feature, round(importance, 5)) for feature, importance in zip(selected_features_names, importances_mean)]
    feature_importances = sorted(feature_importances, key=lambda x:x[1], reverse=True)
    feature_importances_df = pd.DataFrame(feature_importances, columns=['Feature', 'Importance'])
    feature_importances_df['Variance'] = importances_var
    
    feature_importances_df.to_csv(feature_importances_file, index=False)
    feature_importances_df

In [ ]:
# list of features sorted from most to least important
sorted_importances = [importance[1] for importance in feature_importances]
# cumulative importance
cumulative_importances = np.cumsum(sorted_importances)

x_values = list(range(len(feature_importances)))
x_values = [x+1 for x in x_values]

fig = plt.figure(figsize=(4, 4))
ax2 = fig.add_subplot(111)
ax1 = ax2.twinx()
ax2.set_facecolor('whitesmoke')

ax2.bar(x_values, sorted_importances, edgecolor='gray', facecolor='LightSalmon', width=1, alpha=0.5)
ax1.plot(x_values, cumulative_importances, 'k--')

plt.xlim(0.5, len(cumulative_importances)+0.5)

ax1.set_ylim(0, 1.05)

ax2.set_ylabel('Feature Importance')
ax1.set_ylabel('Cumulative Importance')

In [ ]:
# print significant features above some threshold
feature_importances.sort(key=lambda x:x[1])
ft_imps = [x[1] for x in feature_importances]

fig, ax = plt.subplots()
ax.set_facecolor('whitesmoke')
bar = ax.barh(range(len(ft_imps)), ft_imps)

def gradientbars(bars, data):
    ax = bars[0].axes
    lim = ax.get_xlim()+ax.get_ylim()
    for bar in bars:
        bar.set_zorder(1)
        bar.set_facecolor('none')
        bar.set_edgecolor('black')
        x, y = bar.get_xy()
        w, h = bar.get_width(), bar.get_height()
        cmap = plt.get_cmap('coolwarm')
        grad = np.atleast_2d(np.linspace(0, 1*w/max(data), 256))
        ax.imshow(grad, extent=[x, x+w, y, y+h], aspect='auto', zorder=0, norm=mpl.colors.NoNorm(vmin=0, vmax=1), cmap=cmap, alpha=0.8)
        manual_labels = [x[0] for x in feature_importances]
        ax.set_yticks(np.arange(0, len(data), 1).tolist())
        ax.set_yticklabels(manual_labels, minor=False)
    ax.axis(lim)
    ax.set_xlabel('Feature Importance')

gradientbars(bar, ft_imps)
plt.gca().yaxis.grid(False)

# plt.savefig(
#     f'./figures/muller2019/importances.png',
#     bbox_inches='tight',
#     pad_inches=0.1,
#     dpi=150
#     )

plt.show()

In [ ]:
Xy_train_df = pd.read_csv(ml_output_dir + 'Xy_train_muller2019.csv', index_col=False)

for i in range(len(labels_estimated)):
    if Xy_train_df['label'][i] == 1:
        labels_estimated[i] = 1

Xy_train_postpul = Xy_train_df.copy()
Xy_train_postpul['label'] = labels_estimated

@interact
def show_map(feature=Xy_train_postpul.columns):
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.set_facecolor('whitesmoke')
    ax2 = ax1.twiny()
    ax3 = ax2.twinx()
    
    ax1.hist(Xy_train_original_df[feature], bins=25, alpha=0.0)

    h1 = ax2.hist(Xy_train_postpul.loc[Xy_train_postpul['label']==1][feature], bins=5, color='mediumseagreen', label='Positive', alpha=0.4, zorder=3)
    h2 = ax2.hist(Xy_train_postpul.loc[Xy_train_postpul['label']==2][feature], bins=10, color='lightsalmon', label='Synthetic positive', alpha=0.8, zorder=2)
    h3 = ax2.hist(Xy_train_postpul.loc[Xy_train_postpul['label']==0][feature], bins=30, color='silver', label='Negative', zorder= 1)
    
    kde_x = np.linspace(Xy_train_postpul[feature].min(), Xy_train_postpul[feature].max(), 100)
    
    kde1 = stats.gaussian_kde(Xy_train_postpul.loc[Xy_train_postpul['label']==0][feature])
    kde2 = stats.gaussian_kde(Xy_train_postpul.loc[Xy_train_postpul['label']==2][feature])
    kde3 = stats.gaussian_kde(Xy_train_postpul.loc[Xy_train_postpul['label']==1][feature])
    
    k1 = ax3.plot(kde_x, kde1(kde_x), color='grey')
    k2 = ax3.plot(kde_x, kde2(kde_x), color='darkorange')
    k3 = ax3.plot(kde_x, kde3(kde_x), color='green')

    ax2.legend(loc='best')

    ax1.set_xlabel('Total deep-sea sediment thickness (m)')
    ax2.set_xlabel('Total deep-sea sediment thickness' + ' (standardised)')
    ax1.set_ylabel('Frequency')
    ax3.set_ylabel('Probability density')
    
    ax2.set_ylim(0, 38)
    ax2.set_yticks(np.arange(0, 38, 5))
    ax3.set_ylim(0, 2.4)

#     plt.savefig(
#         f'./figures/muller2019/total_sediment_thick_m.png',
#         bbox_inches='tight',
#         pad_inches=0.1,
#         dpi=150
#     )

In [ ]:
selected_features_modified = Xy_train_postpul.columns.drop('label')
# push columns by one
selected_features_modified_1 = deque(selected_features_modified)
selected_features_modified_1.rotate()
selected_features_modified_2 = deque(selected_features_modified_1)
selected_features_modified_2.rotate()

@interact
def show_map(feature1=selected_features_modified, feature2=selected_features_modified_1, feature3=selected_features_modified_2):

    fig = plt.figure(figsize=(6, 6))
    gs = GridSpec(2, 2, hspace=0.4, wspace=0.2, height_ratios=[1, 0.03])
    ax1 = fig.add_subplot(gs[0, :])
    ax2 = ax1.twinx()
    ax3 = ax2.twiny()
    
    min1 = Xy_train_original_df[feature1].min()
    max1 = Xy_train_original_df[feature1].max()
    min2 = Xy_train_original_df[feature2].min()
    max2 = Xy_train_original_df[feature2].max()

    x_range = np.linspace(min1, max1+0.1*max1, num=100)
    y_range = np.linspace(min2, max2+0.1*max2, num=100)
    grid_x, grid_y = np.meshgrid(x_range, y_range)
    
    grid_data = griddata(list(zip(Xy_train_original_df[feature1], Xy_train_original_df[feature2])), Xy_train_original_df[feature3],
                         (grid_x, grid_y), method='nearest', fill_value=0)
    grid_data = ndimage.gaussian_filter(grid_data, sigma=3)
    cb1 = ax1.imshow(grid_data.T, extent=(min1, max1, min2, max2), origin='lower', aspect='auto', cmap=plt.cm.Spectral_r, alpha=0)

    sc1 = ax1.scatter(Xy_train_original_df.loc[Xy_train_original_df['label']==0, feature1],
                      Xy_train_original_df.loc[Xy_train_original_df['label']==0, feature2], 40, marker='.', c='blue', alpha=0)
    sc2 = ax1.scatter(Xy_train_original_df.loc[Xy_train_original_df['label']==1, feature1],
                      Xy_train_original_df.loc[Xy_train_original_df['label']==1, feature2], 40, marker='.', c='orange', alpha=0)

    min1 = Xy_train_postpul[feature1].min()
    max1 = Xy_train_postpul[feature1].max()
    min2 = Xy_train_postpul[feature2].min()
    max2 = Xy_train_postpul[feature2].max()
    
    x_range = np.linspace(min1, max1+0.1*max1, num=100)
    y_range = np.linspace(min2, max2+0.1*max2, num=100)
    grid_x, grid_y = np.meshgrid(x_range, y_range)
    
    grid_data = griddata(list(zip(Xy_train_postpul[feature1], Xy_train_postpul[feature2])), Xy_train_postpul[feature3],
                         (grid_x, grid_y), method='nearest', fill_value=0)
    grid_data = ndimage.gaussian_filter(grid_data, sigma=3)
    cb2 = ax3.imshow(grid_data.T, extent=(min1, max1, min2, max2), origin='lower', aspect='auto', cmap=plt.cm.Spectral_r, alpha=0.7)
    
    sc3 = ax3.scatter(Xy_train_postpul.loc[Xy_train_postpul['label']==1, feature1],
                      Xy_train_postpul.loc[Xy_train_postpul['label']==1, feature2],
                      100, marker='.', facecolor='mediumseagreen', edgecolor='black', zorder=3)
    sc4 = ax3.scatter(Xy_train_postpul.loc[Xy_train_postpul['label']==2, feature1],
                      Xy_train_postpul.loc[Xy_train_postpul['label']==2, feature2],
                      100, marker='.', facecolor='orangered', edgecolor='black', alpha=0.7, zorder=2)
    sc5 = ax3.scatter(Xy_train_postpul.loc[Xy_train_postpul['label']==0, feature1],
                      Xy_train_postpul.loc[Xy_train_postpul['label']==0, feature2],
                      100, marker='.', facecolor='silver', edgecolor='black', alpha=0.7, zorder=1)
    
    # conv_angle_deg
    ax1.set_xlim(-90, 90)
    ax3.set_xlim(-1.8, 0.8)
    
    ax3.legend([sc3, sc4, sc5], ['Positive', 'Synthetic positive', 'Negative'], loc='best',  borderaxespad=0.1, fontsize=8) # numpoints=1

    ax1.set_xlabel('Obliquity angle of the relative motion vector (deg)')
    ax1.set_ylabel('Length of the arc segment (deg)')
    ax2.set_ylabel('Length of the arc segment (standardised)')
    ax3.set_xlabel('Obliquity angle of the relative motion vector (standardised)')
        
    cax1 = fig.add_subplot(gs[1, 1])
    cax2 = fig.add_subplot(gs[1, 0])
        
    fig.colorbar(cb2, cax=cax2, orientation='horizontal', label='Distance to the nearest\ntrench edge(standardised)', extend='both')
    fig.colorbar(cb1, cax=cax1, orientation='horizontal', label='Distance to the nearest\ntrench edge (deg)', extend='both')
    
#     plt.savefig(
#         f'./figures/muller2019/features_three/dist_nearest_edge_deg.png',
#         bbox_inches='tight',
#         pad_inches=0.1,
#         dpi=150
#     )

    plt.show()

In [ ]:
Xy_train_postpul_pivot = Xy_train_postpul.pivot(columns=['label'])
Xy_train_original_df_pivot = Xy_train_original_df.pivot(columns=['label'])

nb_groups1 = Xy_train_postpul['label'].nunique()
nb_groups2 = Xy_train_original_df['label'].nunique()

@interact
def show_map(feature=Xy_train_postpul.columns):
    bplot1 = [Xy_train_postpul_pivot[feature][var].dropna() for var in Xy_train_postpul_pivot[feature]]
    bplot2 = [Xy_train_original_df_pivot[feature][var].dropna() for var in Xy_train_original_df_pivot[feature]]

    fig, ax1 = plt.subplots(figsize=(9, 6))
    box_param = dict(whis=(5, 95), widths=0.2, patch_artist=True,
                     flierprops=dict(marker='.', markeredgecolor='black',
                     fillstyle=None), medianprops=dict(color='black'))

    space = 0.15
    ax1.boxplot(bplot1, positions=np.arange(nb_groups1)-space,
                boxprops=dict(facecolor='tab:blue'), **box_param)

    ax2 = ax1.twinx()
    ax2.boxplot(bplot2, positions=np.arange(nb_groups2)+space,
                boxprops=dict(facecolor='tab:orange'), **box_param)

    # format x ticks
    labelsize = 12
    ax1.set_xticks(np.arange(nb_groups1))
#     ax1.set_xticklabels([f'{label}' for label in np.sort(Xy_train_postpul['label'].unique())])
    ax1.set_xticklabels(['Negative', 'Positive', 'Synthetic Positive'])
    ax1.tick_params(axis='x', labelsize=labelsize)

    # format y ticks
    yticks_fmt = dict(axis='y', labelsize=labelsize)
    ax1.tick_params(colors='tab:blue', **yticks_fmt)
    ax2.tick_params(colors='tab:orange', **yticks_fmt)

    # format axes labels
    label_fmt = dict(size=12, labelpad=15)
    ax1.set_xlabel(feature, **label_fmt)
    ax1.set_ylabel(feature + '\n(Standardised)', color='tab:blue', **label_fmt)
    ax2.set_ylabel(feature + '\n(Actual)', color='tab:orange', **label_fmt)

    plt.show()

In [ ]:
Xy_train_postpul_pivot = Xy_train_postpul.pivot(columns=['label'])
Xy_train_original_df_pivot = Xy_train_original_df.pivot(columns=['label'])

nb_groups1 = Xy_train_postpul['label'].nunique()
nb_groups2 = Xy_train_original_df['label'].nunique()

@interact
def show_map(feature=Xy_train_postpul.columns):
    bplot1 = [Xy_train_postpul_pivot[feature][var].dropna() for var in Xy_train_postpul_pivot[feature]]
    bplot2 = [Xy_train_original_df_pivot[feature][var].dropna() for var in Xy_train_original_df_pivot[feature]]

    fig, ax1 = plt.subplots(figsize=(5, 5))
    box_param1 = dict(whis=(5, 95), widths=0.2, patch_artist=True,
                      flierprops=dict(marker='.', markeredgecolor='black', fillstyle=None),
                      medianprops=dict(color='black'), boxprops=dict(facecolor='tab:blue'))
    box_param2 = dict(whis=(5, 95), widths=0, patch_artist=True,
                      flierprops=dict(marker='.', markeredgecolor='none', fillstyle=None),
                      medianprops=dict(color='none'), whiskerprops=dict(color='none'),
                      boxprops=dict(facecolor='none', edgecolor='none'))

    ax1.boxplot(bplot1, positions=np.arange(nb_groups1), **box_param1)

    ax2 = ax1.twinx()
    ax2.boxplot(bplot2, positions=np.arange(nb_groups2), **box_param2)

    # format x ticks
    labelsize = 12
    ax1.set_xticks(np.arange(nb_groups1))
    ax1.set_xticklabels(['Negative', 'Positive', 'Synthetic Positive'])
    ax1.tick_params(axis='x', labelsize=labelsize)

    # format y ticks
    yticks_fmt = dict(axis='y', labelsize=labelsize)

    # format axes labels
    label_fmt = dict(size=12, labelpad=15)
    ax1.set_xlabel(feature, **label_fmt)
    ax1.set_ylabel(feature + '\n(Standardised)', **label_fmt)
    ax2.set_ylabel(feature + '\n(Actual)', **label_fmt)

    plt.show()

In [ ]:
# function to calculate whiskers (for outliers)
def calculate_whiskers(data):
    q1, q3 = np.percentile(data, [25, 75])
    iqr = q3 - q1
    whisker_low = q1 - 1.5 * iqr
    whisker_high = q3 + 1.5 * iqr
    return whisker_low, whisker_high

# pivot dataframes
Xy_train_postpul_pivot = Xy_train_postpul.pivot(columns=['label'])
Xy_train_original_df_pivot = Xy_train_original_df.pivot(columns=['label'])

# calculate the number of unique groups
nb_groups1 = Xy_train_postpul['label'].nunique()
nb_groups2 = Xy_train_original_df['label'].nunique()

colors = ['grey', 'seagreen', 'tomato']

@interact
def show_map(feature=Xy_train_postpul.columns):
    vplot1_data = [Xy_train_postpul_pivot[feature][var].dropna() for var in Xy_train_postpul_pivot[feature]]
    vplot2_data = [Xy_train_original_df_pivot[feature][var].dropna() for var in Xy_train_original_df_pivot[feature]]

    fig, ax1 = plt.subplots()
    ax1.set_facecolor('whitesmoke')

    # create violin plots
    vplot1_parts = ax1.violinplot(vplot1_data, positions=np.arange(nb_groups1))
    
    for i, part in enumerate(vplot1_parts['bodies']):
        part.set_facecolor(colors[i])
        part.set_edgecolor(colors[i])
        
    vplot1_parts['cbars'].set_edgecolor('black')
    vplot1_parts['cmins'].set_edgecolor('black')
    vplot1_parts['cmaxes'].set_edgecolor('black')
    
    ax2 = ax1.twinx()
    vplot2_parts = ax2.violinplot(vplot2_data, positions=np.arange(nb_groups2))
    
    for part in vplot2_parts['bodies']:
        part.set_facecolor('none')
        part.set_edgecolor('none')
        
    vplot2_parts['cbars'].set_edgecolor('none')
    vplot2_parts['cmins'].set_edgecolor('none')
    vplot2_parts['cmaxes'].set_edgecolor('none')

    # plotting outliers for the first violin plot
    for i, data in enumerate(vplot1_data):
        low, high = calculate_whiskers(data)
        outliers = data[(data > high) | (data < low)]
        ax1.scatter([i]*len(outliers), outliers, facecolor='red', edgecolor='black', s=20, zorder=2)

    # format x ticks
    labelsize = 12
    ax1.set_xticks(np.arange(nb_groups1))
    ax1.set_xticklabels(['Negative', 'Positive', 'Synthetic positive'])

    # format axes labels
    ax1.set_ylabel('Obliquity angle of the overriding absolute\nplate velocity (standardised)')
    ax2.set_ylabel('Obliquity angle of the overriding absolute\nplate velocity (cm/yr)')
    
#     plt.savefig(
#         f'./figures/muller2019/trench_abs_angle_deg.png',
#         bbox_inches='tight',
#         pad_inches=0.1,
#         dpi=150
#     )
    
    plt.show()

In [ ]:
conv_angle_deg_postpul = Xy_train_postpul[['conv_angle_deg', 'label']]
conv_angle_deg_postpul_filtered = conv_angle_deg_postpul[(conv_angle_deg_postpul['conv_angle_deg'] <= 0.8) & (conv_angle_deg_postpul['conv_angle_deg'] >= -1.8)]

conv_angle_deg_original = Xy_train_original_df[['conv_angle_deg', 'label']]
conv_angle_deg_original_filtered = conv_angle_deg_original[(conv_angle_deg_original['conv_angle_deg'] <= 90) & (conv_angle_deg_original['conv_angle_deg'] >= -90)]

# pivot dataframes
conv_angle_deg_postpul_pivot = conv_angle_deg_postpul_filtered.pivot(columns=['label'])
conv_angle_deg_original_pivot = conv_angle_deg_original_filtered.pivot(columns=['label'])

# calculate the number of unique groups
nb_groups1 = conv_angle_deg_postpul_filtered['label'].nunique()
nb_groups2 = conv_angle_deg_original_filtered['label'].nunique()

vplot1_data = [conv_angle_deg_postpul_pivot['conv_angle_deg'][var].dropna() for var in conv_angle_deg_postpul_pivot['conv_angle_deg']]
vplot2_data = [conv_angle_deg_original_pivot['conv_angle_deg'][var].dropna() for var in conv_angle_deg_original_pivot['conv_angle_deg']]

fig, ax1 = plt.subplots()
ax1.set_facecolor('whitesmoke')

# create violin plots
vplot1_parts = ax1.violinplot(vplot1_data, positions=np.arange(nb_groups1))

for i, part in enumerate(vplot1_parts['bodies']):
    part.set_facecolor(colors[i])
    part.set_edgecolor(colors[i])

vplot1_parts['cbars'].set_edgecolor('black')
vplot1_parts['cmins'].set_edgecolor('black')
vplot1_parts['cmaxes'].set_edgecolor('black')

ax2 = ax1.twinx()
vplot2_parts = ax2.violinplot(vplot2_data, positions=np.arange(nb_groups2))

for part in vplot2_parts['bodies']:
    part.set_facecolor('none')
    part.set_edgecolor('none')

vplot2_parts['cbars'].set_edgecolor('none')
vplot2_parts['cmins'].set_edgecolor('none')
vplot2_parts['cmaxes'].set_edgecolor('none')

# plotting outliers for the first violin plot
for i, data in enumerate(vplot1_data):
    low, high = calculate_whiskers(data)
    outliers = data[(data > high) | (data < low)]
    ax1.scatter([i]*len(outliers), outliers, facecolor='red', edgecolor='black', s=20, zorder=2)

# format x ticks
labelsize = 12
ax1.set_xticks(np.arange(nb_groups1))
ax1.set_xticklabels(['Negative', 'Positive', 'Synthetic positive'])

# format axes labels
ax1.set_ylabel('Obliquity angle of the relative motion vector\n(standardised)')
ax2.set_ylabel('Obliquity angle of the relative motion vector (deg)')

# plt.savefig(
#     f'./figures/muller2019/conv_angle_deg.png',
#     bbox_inches='tight',
#     pad_inches=0.1,
#     dpi=150
# )

plt.show()

### Probability

In [ ]:
target_points_prob_files_lst = []
target_points_prob_lst = []

for time in time_steps:
    target_points_prob_files_lst.append(ml_output_dir + f'target_points_prob_{time}_Ma.csv')

for i, target_points_prob_file in tqdm(enumerate(target_points_prob_files_lst), total=len(target_points_prob_files_lst)):
    if not(os.path.isfile(target_points_prob_file)):
        df = target_points_ml_in_lst[target_points_prob_files_lst.index(target_points_prob_file)]
        probs = bc_best.predict_proba(df)[:, 1].reshape(-1, 1)

        mm_scaler1 = MinMaxScaler()
        probs_scaled = mm_scaler1.fit_transform(probs)

        df_xy = df.copy()
        df_xy['lon'] = target_points_coreg_out_lst[i]['lon'].to_numpy()
        df_xy['lat'] = target_points_coreg_out_lst[i]['lat'].to_numpy()
        df_xy['prob'] = probs_scaled
        df_xy.to_csv(target_points_prob_file, index=False)
        target_points_prob_lst.append(df_xy)
        
for target_points_prob_file in target_points_prob_files_lst:
    target_points_prob_lst.append(pd.read_csv(target_points_prob_file, index_col=False))

In [ ]:
proj = ccrs.LambertAzimuthalEqualArea(150, 0)

@interact
def show_map(time=time_steps):
    lons_lats_recon = []
    
    for min_occ in min_occ_data.iterrows():
        if time == 0:
            lons_lats_recon.append((min_occ[1]['lon'], min_occ[1]['lat']))
        elif int(min_occ[1]['age']) < time:
            lons_lats_recon.append((np.nan, np.nan))
        elif int(min_occ[1]['age']) == time:
            lons_lats_recon.append((min_occ[1]['lon_recon'], min_occ[1]['lat_recon']))
        else:
            lat_lon_recon = get_recon_ccords([min_occ[1]['lon']],
                                             [min_occ[1]['lat']],
                                             plate_motion_model='muller2019', # assign 'muller2019' or 'muller2016'
                                             time=time)[0]
            lons_lats_recon.append(tuple((lat_lon_recon[1], lat_lon_recon[0])))
            
    # call the PlotTopologies object
    gplot = gplately.PlotTopologies(model, coastlines, continents, cob, time=time)
    
    if plate_motion_model == 'muller2016':
        agegrid_file = agegrid_dir + f'Muller_etal_2016_AREPS_v1.17_AgeGrid-{time}.nc'
    elif plate_motion_model == 'muller2019':
        agegrid_file = agegrid_dir + f'Muller_etal_2019_Tectonics_v2.0_AgeGrid-{time}.nc'

    agegrid = gplately.grids.read_netcdf_grid(agegrid_file)
    
    plot_x = target_points_prob_lst[time_steps.index(time)]['lon']
    plot_y = target_points_prob_lst[time_steps.index(time)]['lat']
    
    # dual colour bars
    fig = plt.figure(figsize=(6, 8))
    gs = GridSpec(2, 2, hspace=-0.1, wspace=0.1, height_ratios=[1, 0.02])
    ax = fig.add_subplot(gs[0, :], projection=proj)
    
    set_ax(ax, target_extent, 15, 15, stock_img=False, order=9)

    im = gplot.plot_grid(ax, agegrid.data, cmap='viridis', vmin=0, vmax=230, alpha=0.5, zorder=1)

    gplot.plot_continents(ax, edgecolor='none', facecolor='tan', alpha=0.7, zorder=2)
    gplot.plot_ridges(ax, color='red', label='Ridge', alpha=0.5, zorder=3)
    gplot.plot_plate_motion_vectors(ax, spacingX=20, spacingY=20, normalise=False, regrid_shape=20, alpha=0.2, zorder=4)
    
    sc0 = ax.scatter(
        plot_x,
        plot_y,
        transform=ccrs.PlateCarree(),
        marker='.',
        c=target_points_prob_lst[time_steps.index(time)]['prob'],
        s=30,
        cmap='YlOrRd',
        zorder=5
    )
    
    sc1 = ax.scatter(
        [coords[0] for coords in lons_lats_recon],
        [coords[1] for coords in lons_lats_recon],
        transform=ccrs.PlateCarree(),
        marker='X',
        facecolor='yellow',
        edgecolor='black',
        s=50,
        alpha=0.7,
        zorder=6
    )

    gplot.plot_trenches(ax, color='k', alpha=0.3, zorder=7)
    gplot.plot_subduction_teeth(ax, spacing=0.03, color='k', alpha=0.3, zorder=8)

    cax1 = fig.add_subplot(gs[1, 0])
    cax2 = fig.add_subplot(gs[1, 1])
    
    fig.colorbar(sc0, cax=cax2, orientation='horizontal', label='Mineralisation probability')
    fig.colorbar(im, cax=cax1, orientation='horizontal', label='Seafloor age (Ma)', extend='max')
    
    ax.set_title(f'Porphyry Cu-Au Mineralisation Probability {time} Ma')
    ax.legend(loc='lower left')
        
    plt.show()

In [ ]:
mask_coords_lst = []
for mask_coords_file in mask_coords_files_lst:
    mask_coords_lst.append(pd.read_csv(mask_coords_file, index_col=False))

proj = ccrs.LambertAzimuthalEqualArea(150, 0)

@interact
def show_map(time=time_steps):
    lons_lats_recon = []
    
    for min_occ in min_occ_data.iterrows():
        if time == 0:
            lons_lats_recon.append((min_occ[1]['lon'], min_occ[1]['lat']))
        elif int(min_occ[1]['age']) < time:
            lons_lats_recon.append((np.nan, np.nan))
        elif int(min_occ[1]['age']) == time:
            lons_lats_recon.append((min_occ[1]['lon_recon'], min_occ[1]['lat_recon']))
        else:
            lat_lon_recon = get_recon_ccords([min_occ[1]['lon']],
                                             [min_occ[1]['lat']],
                                             plate_motion_model='muller2019', # assign 'muller2019' or 'muller2016'
                                             time=time)[0]
            lons_lats_recon.append(tuple((lat_lon_recon[1], lat_lon_recon[0])))
    
    # call the PlotTopologies object
    gplot = gplately.PlotTopologies(model, coastlines, continents, cob, time=time)
    
    if plate_motion_model == 'muller2016':
        agegrid_file = agegrid_dir + f'Muller_etal_2016_AREPS_v1.17_AgeGrid-{time}.nc'
    elif plate_motion_model == 'muller2019':
        agegrid_file = agegrid_dir + f'Muller_etal_2019_Tectonics_v2.0_AgeGrid-{time}.nc'

    agegrid = gplately.grids.read_netcdf_grid(agegrid_file)
    
    mask_coords = mask_coords_lst[time_steps.index(time)]
    
    probabilities = []
    count = 0
            
    for mask in mask_coords['include']:
        if mask:
            probabilities.append(target_points_prob_lst[time_steps.index(time)]['prob'][count])
            count += 1
        else:
            probabilities.append(np.nan)
    
    nx = mask_coords_lst[time_steps.index(time)]['lon'].nunique()
    ny = mask_coords_lst[time_steps.index(time)]['lat'].nunique()
    
    x_min = mask_coords_lst[time_steps.index(time)]['lon'].min()
    x_max = mask_coords_lst[time_steps.index(time)]['lon'].max()
    y_min = mask_coords_lst[time_steps.index(time)]['lat'].min()
    y_max = mask_coords_lst[time_steps.index(time)]['lat'].max()
    
    probabilities_2d = np.reshape(probabilities, (ny, nx))
    probabilities_2d_ud = np.flipud(np.reshape(probabilities, (ny, nx)))
    
    # dual colour bars
    fig = plt.figure(figsize=(6, 8))
    gs = GridSpec(2, 2, hspace=-0.1, wspace=0.1, height_ratios=[1, 0.02])
    ax = fig.add_subplot(gs[0, :], projection=proj)
    
    set_ax(ax, target_extent, 15, 15, stock_img=False, order=9)

    im0 = gplot.plot_grid(ax, agegrid.data, cmap='viridis', vmin=0, vmax=230, alpha=0.5, zorder=1)

    gplot.plot_continents(ax, edgecolor='none', facecolor='tan', alpha=0.7, zorder=2)
    gplot.plot_ridges(ax, color='red', label='Ridge', alpha=0.5, zorder=3)
    gplot.plot_plate_motion_vectors(ax, spacingX=20, spacingY=20, normalise=False, regrid_shape=20, alpha=0.2, zorder=4)
    
    im1 = plt.imshow(
        probabilities_2d,
        transform=ccrs.PlateCarree(),
        origin='lower',
        cmap='YlOrRd',
        interpolation='bilinear',
        extent=(x_min, x_max, y_min, y_max),
        zorder=5
    )
    
    sc1 = ax.scatter(
        [coords[0] for coords in lons_lats_recon],
        [coords[1] for coords in lons_lats_recon],
        transform=ccrs.PlateCarree(),
        marker='X',
        facecolor='yellow',
        edgecolor='black',
        s=50,
        alpha=0.7,
        zorder=6
    )

    gplot.plot_trenches(ax, color='k', alpha=0.3, zorder=7)
    gplot.plot_subduction_teeth(ax, spacing=0.03, color='k', alpha=0.3, zorder=8)
    
    cax1 = fig.add_subplot(gs[1, 0])
    cax2 = fig.add_subplot(gs[1, 1])
    
    fig.colorbar(im1, cax=cax2, orientation='horizontal', label='Mineralisation probability')
    fig.colorbar(im0, cax=cax1, orientation='horizontal', label='Seafloor age (Ma)', extend='max')
    
    ax.set_title(f'Porphyry Cu-Au Mineralisation Probability {time} Ma')
    ax.legend(loc='lower left')
        
    plt.show()

# Coregistration of Mineral Occurrences

In [ ]:
min_occ_prob_dir = parameters['min_occ_prob_dir']
min_occ_prob_files_lst = []
min_occ_prob_tran_files_lst = []

for index in min_occ_data['index']:
    min_occ_prob_files_lst.append(min_occ_prob_dir + f'min_occ_features_{index}.csv')
    min_occ_prob_tran_files_lst.append(min_occ_prob_dir + f'min_occ_features_tran_{index}.csv')

coregistration_point(
    min_occ_data,
    conv_dir,
    conv_prefix,
    conv_ext,
    min_occ_prob_dir,
    file_prefix='min_occ_features',
    time_steps=time_steps,
    search_radius=3,
    plate_motion_model='muller2019' # assign 'muller2019' or 'muller2016'
)

for min_occ_prob_file, min_occ_prob_tran_file in zip(min_occ_prob_files_lst, min_occ_prob_tran_files_lst):
    if not os.path.isfile(min_occ_prob_tran_file):
        min_occ_prob = pd.read_csv(min_occ_prob_file, index_col=False)
        min_occ_prob_tran = min_occ_prob.copy()
        probs = []

        for i, row in min_occ_prob.iterrows():
            try:
                row_features = row[features_list]
            except:
                Xy_train_original_df = pd.read_csv(ml_input_dir + 'features_labels_original.csv', index_col=False)
                features_list = Xy_train_original_df.columns.to_list()
                features_list.remove('label')
                row_features = row[features_list]

            if row_features.isnull().values.any():
                probs.append(np.nan)
            else:
                try:
                    row_features = st_scaler.transform(row_features.values.reshape(1, -1))
                except:
                    # load the model
                    with open(ml_input_dir + 'st_scaler.pkl', 'rb') as f:
                        st_scaler = pickle.load(f)
                    row_features = st_scaler.transform(row_features.values.reshape(1, -1))

                min_occ_prob_tran.loc[min_occ_prob_tran['age'] == i, features_list] = row_features[0].tolist()
                prob = bc_best.predict_proba(row_features)[0, 1]
                probs.append(prob)

        mm_scaler2 = MinMaxScaler()
        probs_scaled = mm_scaler2.fit_transform(np.array(probs).reshape(-1, 1))
        min_occ_prob['prob'] = probs_scaled
        min_occ_prob_tran['prob'] = probs_scaled
        min_occ_prob.to_csv(min_occ_prob_file, index=False)
        min_occ_prob_tran.to_csv(min_occ_prob_tran_file, index=False)

In [ ]:
Xy_train_original_df = pd.read_csv(ml_input_dir + 'features_labels_original.csv', index_col=False)
features_list = Xy_train_original_df.columns.to_list()
features_list.remove('label')

def smooth_1D(array, std=1):
    return gaussian_filter1d(array, std)

@interact
def show_map(file1=min_occ_prob_files_lst, file2=min_occ_prob_tran_files_lst, feature=features_list):
    df1 = pd.read_csv(file1, index_col=False)
    df2 = pd.read_csv(file2, index_col=False)

    fig = plt.figure(figsize=(12, 4))
    ax2 = fig.add_subplot(121, xlim=[df1['age'].max(), 0])
    ax1 = ax2.twinx()

    ax2.plot(df1['age'], df1['prob'], c='red')
    
    index1 = min_occ_prob_files_lst.index(file1)
    age1 = min_occ_data.iloc[index1]['age']
    ax2.vlines(x=age1, ymin=0, ymax=1, color='k', linestyles=':')
    
    ax2.set_ylim(0, 1)
    ax1.plot(df1['age'], df1[feature], c='blue')
    
    ax4 = fig.add_subplot(122, xlim=[df2['age'].max(), 0])
    ax3 = ax4.twinx()

    ax4.plot(df2['age'], df2['prob'], c='red')
    
    index2 = min_occ_prob_tran_files_lst.index(file2)
    age2 = min_occ_data.iloc[index2]['age']
    ax4.vlines(x=age2, ymin=0, ymax=1, color='k', linestyles=':')
    
    ax4.set_ylim(0, 1)
    ax3.plot(df2['age'], df2[feature], c='blue')

    ax2.set_ylabel('Probability')
    ax1.set_ylabel(feature + ' (Actual)')
    ax4.set_ylabel('Probability')
    ax3.set_ylabel(feature + ' (Standardised)')
    
    fig.tight_layout()
    
    plt.show()

In [ ]:
# smoothing function for the plots
def smooth_data(x, y, points=200):
    x_new = np.linspace(x.min(), x.max(), points)
    spl = make_interp_spline(x, y, k=3)  # b-spline
    y_smooth = spl(x_new)
    return x_new, y_smooth

@interact
def show_map(file1=min_occ_prob_files_lst, feature=features_list):
    df1 = pd.read_csv(file1, index_col=False)

    fig = plt.figure(figsize=(6, 4))
    ax1 = fig.add_subplot(111, xlim=[df1['age'].max(), 0])
    ax2 = ax1.twinx()
    
    ax1.set_facecolor('whitesmoke')

    # smooth and plot probability
    age_smooth, prob_smooth = smooth_data(df1['age'], df1['prob'])
    ln1 = ax1.plot(age_smooth, prob_smooth, c='orangered', label='Probability')

    index1 = min_occ_prob_files_lst.index(file1)
    age1 = min_occ_data.iloc[index1]['age']
    ln2 = ax1.vlines(x=age1, ymin=0, ymax=1, color='k', linestyles=':', label='Age of formation')

    ax1.set_ylim(0, 1)

    # smooth and plot feature
    age_smooth, feature_smooth = smooth_data(df1['age'], df1[feature])
    ln3 = ax2.plot(age_smooth, feature_smooth, c='royalblue', label='Feature')
    
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    lines = [lines1[0], lines2[0], lines1[1]]
    labels = [labels1[0], labels2[0], labels1[1]]
    ax1.legend(lines, labels, loc='lower left')

    ax1.set_xlabel('Age (Ma)')
    ax1.set_ylabel('Mineralisation probability')
    ax2.set_ylabel('Seafloor spreading rate (mm/yr)')
        
    fig.tight_layout()
    
#     plt.savefig(
#         f'./figures/muller2019/seafloor_spread_rate_mm_yr.png',
#         bbox_inches='tight',
#         pad_inches=0.1,
#         dpi=150
#     )
        
    plt.show()

In [ ]:
proj = ccrs.LambertAzimuthalEqualArea(150, 0)

@interact
def show_map(file=min_occ_prob_files_lst, time=time_steps):
    df = pd.read_csv(file, index_col=False)
    lons = df.loc[df['age'] >= time]['lon'].tolist()
    lats = df.loc[df['age'] >= time]['lat'].tolist()
    bm = df.loc[df['age'] >= time]['before_mineralisation'].tolist()
    val = df.loc[df['age'] >= time]['valid'].tolist()
    
    lons_inval = []
    lats_inval = []
    lons_bm = []
    lats_bm = []
    lons_am = []
    lats_am = []
    
    # colour of the last point
    if not val[0]:
        last_point = 'invalid'
    elif bm[0]:
        last_point = 'before_mineralisation'
    else:
        last_point = 'after_mineralisation'
    
    for index in range(1, len(lons)):
        if not val[index]:
            lons_inval.append(lons[index])
            lats_inval.append(lats[index])
        elif bm[index]:
            lons_bm.append(lons[index])
            lats_bm.append(lats[index])
        else:
            lons_am.append(lons[index])
            lats_am.append(lats[index])
        
    # call the PlotTopologies object
    gplot = gplately.PlotTopologies(model, coastlines, continents, cob, time=time)

    if plate_motion_model == 'muller2016':
        agegrid_file = agegrid_dir + f'Muller_etal_2016_AREPS_v1.17_AgeGrid-{time}.nc'
    elif plate_motion_model == 'muller2019':
        agegrid_file = agegrid_dir + f'Muller_etal_2019_Tectonics_v2.0_AgeGrid-{time}.nc'
        
    agegrid = gplately.grids.read_netcdf_grid(agegrid_file)
    
    mask_coords = mask_coords_lst[time_steps.index(time)]
    
    probabilities = []
    count = 0
            
    for mask in mask_coords['include']:
        if mask:
            probabilities.append(target_points_prob_lst[time_steps.index(time)]['prob'][count])
            count += 1
        else:
            probabilities.append(np.nan)
    
    nx = mask_coords_lst[time_steps.index(time)]['lon'].nunique()
    ny = mask_coords_lst[time_steps.index(time)]['lat'].nunique()
    
    x_min = mask_coords_lst[time_steps.index(time)]['lon'].min()
    x_max = mask_coords_lst[time_steps.index(time)]['lon'].max()
    y_min = mask_coords_lst[time_steps.index(time)]['lat'].min()
    y_max = mask_coords_lst[time_steps.index(time)]['lat'].max()
    
    probabilities_2d = np.reshape(probabilities, (ny, nx))
    probabilities_2d_ud = np.flipud(np.reshape(probabilities, (ny, nx)))
    
    # single colour bar
    fig = plt.figure(figsize=(6, 8))
    ax = plt.axes(projection=proj)
    
    set_ax(ax, target_extent, 15, 15, stock_img=False, order=10)

    im0 = gplot.plot_grid(ax, agegrid.data, cmap='Blues', vmin=0, vmax=230, alpha=0.7, zorder=1)
    
    gplot.plot_continents(ax, edgecolor='none', facecolor='tan', zorder=2)
    gplot.plot_ridges_and_transforms(ax, color='red', alpha=0.5, zorder=3)
    gplot.plot_plate_motion_vectors(ax, spacingX=20, spacingY=20, normalise=False, regrid_shape=20, alpha=0.2, zorder=4)
    
    im1 = plt.imshow(
        probabilities_2d,
        transform=ccrs.PlateCarree(),
        origin='lower',
        cmap='Spectral_r',
        interpolation='bilinear',
        extent=(x_min, x_max, y_min, y_max),
        alpha=0.5,
        zorder=5
    )
    
    if last_point == 'invalid':
        sc = ax.scatter(lons[0], lats[0], transform=ccrs.PlateCarree(), marker='*', facecolor='gray', s=20, zorder=6)
    elif last_point == 'before_mineralisation':
        sc = ax.scatter(lons[0], lats[0], transform=ccrs.PlateCarree(), marker='*', facecolor='black', s=20, zorder=6)
    else:
        sc = ax.scatter(lons[0], lats[0], transform=ccrs.PlateCarree(), marker='*', facecolor='red', s=20, zorder=6)
    
    sc = ax.scatter(lons_bm, lats_bm, transform=ccrs.PlateCarree(), marker='.', facecolor='black', s=20, zorder=7)
    sc = ax.scatter(lons_am, lats_am, transform=ccrs.PlateCarree(), marker='.', facecolor='red', s=20, zorder=7)
    sc = ax.scatter(lons_inval, lats_inval, transform=ccrs.PlateCarree(), marker='.', facecolor='gray', s=20, zorder=7)
    
    gplot.plot_trenches(ax, color='k', alpha=0.3, zorder=8)
    gplot.plot_subduction_teeth(ax, spacing=0.03, color='k', alpha=0.3, zorder=9)
    
    fig.colorbar(im1, orientation='horizontal', shrink=0.4, pad=0.05, label='Probability')
    
    plt.show()

In [ ]:
def make_lines_glow(
    ax=None,
    n_glow_lines: int = 10,
    diff_linewidth: float = 1.05,
    alpha_line: float = 0.3,
    lines=None,
) -> None:
    """Add a glow effect to the lines in an axis object.
    Each existing line is redrawn several times with increasing width and low alpha to create the glow effect.
    """
    if not ax:
        ax = plt.gca()

    lines = ax.get_lines() if lines is None else lines
    lines = [lines] if isinstance(lines, Line2D) else lines

    alpha_value = alpha_line / n_glow_lines

    for line in lines:

        data = line.get_data(orig=False)
        linewidth = line.get_linewidth()
        
        try:
            step_type = line.get_drawstyle().split('-')[1]
        except:
            step_type = None

        for n in range(1, n_glow_lines + 1):
            if step_type:
                glow_line, = ax.step(*data)
            else:
                glow_line, = ax.plot(*data)
            glow_line.update_from(line)  # line properties are copied as seen in this solution: https://stackoverflow.com/a/54688412/3240855

            glow_line.set_alpha(alpha_value)
            glow_line.set_linewidth(linewidth + (diff_linewidth * n))
            glow_line.is_glow_line = True  # mark the glow lines, to disregard them in the underglow function.

proj = ccrs.LambertAzimuthalEqualArea(150, 0)

@interact
def show_map(file=min_occ_prob_files_lst, time=time_steps):
    df = pd.read_csv(file, index_col=False)
    lons = df.loc[df['age'] >= time]['lon'].tolist()
    lats = df.loc[df['age'] >= time]['lat'].tolist()
    bm = df.loc[df['age'] >= time]['before_mineralisation'].tolist()
    val = df.loc[df['age'] >= time]['valid'].tolist()
        
    # colour of the last point
    if not val[0]:
        last_point = 'invalid'
    elif bm[0]:
        last_point = 'before_mineralisation'
    else:
        last_point = 'after_mineralisation'

    # list of invalid lines
    lons_inval = []
    lats_inval = []
    lons_inval_temp = []
    lats_inval_temp = []

    for index2 in range(len(lons)-1, -1, -1):
        if not val[index2]:
            lons_inval_temp.append(lons[index2])
            lats_inval_temp.append(lats[index2])
            if index2 == 0:
                lons_inval.append(lons_inval_temp)
                lats_inval.append(lats_inval_temp)
        else:
            if len(lons_inval_temp) != 0:
                lons_inval_temp.append(lons[index2])
                lats_inval_temp.append(lats[index2])
                lons_inval.append(lons_inval_temp)
                lats_inval.append(lats_inval_temp)
                lons_inval_temp = []
                lats_inval_temp = []
            else:
                continue

    # list of lines created before mineralisation
    lons_bm = []
    lats_bm = []
    lons_bm_temp = []
    lats_bm_temp = []

    for index2 in range(len(lons)-1, -1, -1):
        if bm[index2] and val[index2]:
            lons_bm_temp.append(lons[index2])
            lats_bm_temp.append(lats[index2])
            if index2 == 0:
                lons_bm.append(lons_bm_temp)
                lats_bm.append(lats_bm_temp)
        else:
            if len(lons_bm_temp) != 0:
                lons_bm_temp.append(lons[index2])
                lats_bm_temp.append(lats[index2])
                lons_bm.append(lons_bm_temp)
                lats_bm.append(lats_bm_temp)
                lons_bm_temp = []
                lats_bm_temp = []
            else:
                continue

    # list of lines created after mineralisation
    lons_am = []
    lats_am = []
    lons_am_temp = []
    lats_am_temp = []

    for index2 in range(len(lons)-1, -1, -1):
        if not bm[index2] and val[index2]:
            lons_am_temp.append(lons[index2])
            lats_am_temp.append(lats[index2])
            if index2 == 0:
                lons_am.append(lons_am_temp)
                lats_am.append(lats_am_temp)
        else:
            if len(lons_am_temp) != 0:
                lons_am_temp.append(lons[index2])
                lats_am_temp.append(lats[index2])
                lons_am.append(lons_am_temp)
                lats_am.append(lats_am_temp)
                lons_am_temp = []
                lats_am_temp = []
            else:
                continue

    # call the PlotTopologies object
    gplot = gplately.PlotTopologies(model, coastlines, continents, cob, time=time)
    
    if plate_motion_model == 'muller2016':
        agegrid_file = agegrid_dir + f'Muller_etal_2016_AREPS_v1.17_AgeGrid-{time}.nc'
    elif plate_motion_model == 'muller2019':
        agegrid_file = agegrid_dir + f'Muller_etal_2019_Tectonics_v2.0_AgeGrid-{time}.nc'

    agegrid = gplately.grids.read_netcdf_grid(agegrid_file)
    
    mask_coords = mask_coords_lst[time_steps.index(time)]
    
    probabilities = []
    count = 0
            
    for mask in mask_coords['include']:
        if mask:
            probabilities.append(target_points_prob_lst[time_steps.index(time)]['prob'][count])
            count += 1
        else:
            probabilities.append(np.nan)
    
    nx = mask_coords_lst[time_steps.index(time)]['lon'].nunique()
    ny = mask_coords_lst[time_steps.index(time)]['lat'].nunique()
    
    x_min = mask_coords_lst[time_steps.index(time)]['lon'].min()
    x_max = mask_coords_lst[time_steps.index(time)]['lon'].max()
    y_min = mask_coords_lst[time_steps.index(time)]['lat'].min()
    y_max = mask_coords_lst[time_steps.index(time)]['lat'].max()
    
    probabilities_2d = np.reshape(probabilities, (ny, nx))
    probabilities_2d_ud = np.flipud(np.reshape(probabilities, (ny, nx)))
    
    # single colour bar
    fig = plt.figure(figsize=(6, 8))
    ax = plt.axes(projection=proj)
    
    set_ax(ax, target_extent, 15, 15, stock_img=False, order=10)

    im0 = gplot.plot_grid(ax, agegrid.data, cmap='Blues', vmin=0, vmax=230, alpha=0.7, zorder=1)
    
    gplot.plot_continents(ax, edgecolor='none', facecolor='tan', zorder=2)
    gplot.plot_ridges_and_transforms(ax, color='red', alpha=0.5, zorder=3)
    gplot.plot_plate_motion_vectors(ax, spacingX=20, spacingY=20, normalise=False, regrid_shape=20, alpha=0.2, zorder=4)
    
    im1 = plt.imshow(
        probabilities_2d,
        transform=ccrs.PlateCarree(),
        origin='lower',
        cmap='Spectral_r',
        interpolation='bilinear',
        extent=(x_min, x_max, y_min, y_max),
        alpha=0.5,
        zorder=5
    )
        
    for lons_, lats_ in zip(lons_bm, lats_bm):
        sc = ax.plot(lons_, lats_, transform=ccrs.PlateCarree(), color='black', zorder=6)
        
    for lons_, lats_ in zip(lons_am, lats_am):
        sc = ax.plot(lons_, lats_, transform=ccrs.PlateCarree(), color='red', zorder=6)
        
    for lons_, lats_ in zip(lons_inval, lats_inval):
        sc = ax.plot(lons_, lats_, transform=ccrs.PlateCarree(), color='gray', zorder=6)
        
    if last_point == 'invalid':
        sc = ax.scatter(lons[0], lats[0], transform=ccrs.PlateCarree(), marker='*', facecolor='gray', s=20, zorder=7)
    elif last_point == 'before_mineralisation':
        sc = ax.scatter(lons[0], lats[0], transform=ccrs.PlateCarree(), marker='*', facecolor='black', s=20, zorder=7)
    else:
        sc = ax.scatter(lons[0], lats[0], transform=ccrs.PlateCarree(), marker='*', facecolor='red', s=20, zorder=7)
    
    gplot.plot_trenches(ax, color='k', alpha=0.3, zorder=8)
    gplot.plot_subduction_teeth(ax, spacing=0.03, color='k', alpha=0.3, zorder=9)
    
    fig.colorbar(im1, orientation='horizontal', shrink=0.4, pad=0.05, label='Probability')
    
#     make_lines_glow()
    
    plt.show()